<h1>Calibration vs Accuracy Experiment</h1>

Loading Data and Splitting Into Training And Test Sets

In [6]:
#load data

import json

import pandas as pd

# Load the JSON file
with open('../dataset/epl_combined/epl_combined_2015.json', 'r') as f:
    data_2015 = json.load(f)

with open('../dataset/epl_combined/epl_combined_2016.json', 'r') as f:
    data_2016 = json.load(f)

with open('../dataset/epl_combined/epl_combined_2017.json', 'r') as f:
    data_2017 = json.load(f)

with open('../dataset/epl_combined/epl_combined_2018.json', 'r') as f:
    data_2018 = json.load(f)

with open('../dataset/epl_combined/epl_combined_2019.json', 'r') as f:
    data_2019 = json.load(f)

with open('../dataset/epl_combined/epl_combined_2020.json', 'r') as f:
    data_2020 = json.load(f)

with open('../dataset/epl_combined/epl_combined_2021.json', 'r') as f:
    data_2021 = json.load(f)

with open('../dataset/epl_combined/epl_combined_2022.json', 'r') as f:
    data_2022 = json.load(f)

with open('../dataset/epl_combined/epl_combined_2023.json', 'r') as f:
    data_2023 = json.load(f)

# Normalize the data
df_2015 = pd.json_normalize(data_2015)
df_2016 = pd.json_normalize(data_2016)
df_2017 = pd.json_normalize(data_2017)
df_2018 = pd.json_normalize(data_2018)
df_2019 = pd.json_normalize(data_2019)
df_2020 = pd.json_normalize(data_2020)
df_2021 = pd.json_normalize(data_2021)
df_2022 = pd.json_normalize(data_2022)
df_2023 = pd.json_normalize(data_2023) #unseen until betting simulation
#combine the data for 2015-22
df_feature_selection_tuning = pd.concat([df_2015, df_2016, df_2017, df_2018], ignore_index=True)
df_train = pd.concat([df_feature_selection_tuning, df_2019, df_2020, df_2021], ignore_index=True)
df_test = pd.json_normalize(data_2022) #this is the test data. do not combine with the rest
df_final_train = pd.concat([df_train, df_test], ignore_index=True) #this is the final train data
df_betting_simulation = pd.json_normalize(data_2023) #this is the betting simulation data. do not combine with the rest

#Drop the columns that are not needed
#drop 'fixture_id', 'timestamp', 'home_team', 'home_team_id', 'away_team','away_team_id','actual_home_goals', 'actual_away_goals', "home_odds", 'away_odds', 'draw_odds' and break down into x and y
df_feature_selection_tuning = df_feature_selection_tuning.drop(['fixture_id', 'timestamp', 'home_team', 'home_team_id', 'away_team','away_team_id','actual_home_goals', 'actual_away_goals', "home_odds", 'away_odds', 'draw_odds'], axis=1)
df_train = df_train.drop(['fixture_id', 'timestamp', 'home_team', 'home_team_id', 'away_team','away_team_id','actual_home_goals', 'actual_away_goals',"home_odds", 'away_odds', 'draw_odds'], axis=1)
df_test = df_test.drop(['fixture_id', 'timestamp', 'home_team', 'home_team_id', 'away_team','away_team_id','actual_home_goals', 'actual_away_goals',"home_odds", 'away_odds', 'draw_odds'], axis=1)
df_final_train = df_final_train.drop(['fixture_id', 'timestamp', 'home_team', 'home_team_id', 'away_team','away_team_id','actual_home_goals', 'actual_away_goals',"home_odds", 'away_odds', 'draw_odds'], axis=1)
df_betting_simulation = df_betting_simulation.drop(['fixture_id', 'timestamp', 'home_team', 'home_team_id', 'away_team','away_team_id','actual_home_goals', 'actual_away_goals',"home_odds", 'away_odds', 'draw_odds'], axis=1)
#split the data into x and y
y_feature_selection_tuning = df_feature_selection_tuning['actual_winner']
x_feature_selection_tuning = df_feature_selection_tuning.drop(['actual_winner'], axis=1)

y_train = df_train['actual_winner']
x_train = df_train.drop(['actual_winner'], axis=1)

y_test = df_test['actual_winner']
x_test = df_test.drop(['actual_winner'], axis=1)

y_final_train = df_final_train['actual_winner']
x_final_train = df_final_train.drop(['actual_winner'], axis=1)

y_betting_simulation = df_betting_simulation['actual_winner']
x_betting_simulation = df_betting_simulation.drop(['actual_winner'], axis=1)

#encode all y values to 0,1,2 home, away, draw
y_feature_selection_tuning = y_feature_selection_tuning.map({'home': 0, 'away': 1, 'draw': 2})
y_train = y_train.map({'home': 0, 'away': 1, 'draw': 2})
y_test = y_test.map({'home': 0, 'away': 1, 'draw': 2})
y_final_train = y_final_train.map({'home': 0, 'away': 1, 'draw': 2})
y_betting_simulation = y_betting_simulation.map({'home': 0, 'away': 1, 'draw': 2})

Standardising and Removing Highly Correlated Features

In [7]:
#standardise all features of x data frames such that the mean is 0 and the standard deviation is 1
#retain the column names
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_feature_selection_tuning)
x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)
x_feature_selection_tuning = pd.DataFrame(scaler.transform(x_feature_selection_tuning), columns=x_feature_selection_tuning.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)
x_final_train = pd.DataFrame(scaler.transform(x_final_train), columns=x_final_train.columns)
x_betting_simulation = pd.DataFrame(scaler.transform(x_betting_simulation), columns=x_betting_simulation.columns)


""" 
Two features are considered highly correlated if their spearman correlation coefficient is greater than 0.7 Xiao et al. (2016).
 For a given group of highly correlated features, we consider all except the feature which is most correlated with the target to be redundant, 
 and so remove them.

 """
import pandas as pd
import networkx as nx

def remove_redundant_features(X):
    """
    Remove redundant features based on Spearman correlation criteria without target consideration.
    
    Parameters:
    X (pd.DataFrame): Input features DataFrame
    
    Returns:
    pd.DataFrame: DataFrame with redundant features removed
    """
    # Compute feature-feature Spearman correlation matrix
    corr_matrix = X.corr(method='spearman')
    
    # Create graph of features with correlation > 0.7
    G = nx.Graph()
    G.add_nodes_from(corr_matrix.columns)
    features = corr_matrix.columns
    for i in range(len(features)):
        for j in range(i + 1, len(features)):
            if abs(corr_matrix.iloc[i, j]) > 0.7:
                G.add_edge(features[i], features[j])
    
    # Find connected components (groups of correlated features)
    connected_components = list(nx.connected_components(G))
    
    # Identify features to remove
    to_remove = set()
    for group in connected_components:
        if len(group) < 2:
            continue
        
        # Convert set to list for consistent ordering
        group_list = sorted(list(group))
        
        # Keep the first feature and remove others (alternative: keep feature with highest variance)
        # To use highest variance instead, replace next line with:
        # keep_feature = X[group_list].var().idxmax()
        keep_feature = group_list[0]
        
        to_remove.update(feature for feature in group_list if feature != keep_feature)
    
    return X.drop(columns=list(to_remove))

x_feature_selection_tuning = remove_redundant_features(x_feature_selection_tuning)

#drop all the columns that are not in x_feature_selection_tuning
x_train = x_train[x_feature_selection_tuning.columns]
x_test = x_test[x_feature_selection_tuning.columns]

print(f'features: {x_train.columns}')
print(f'number of features: {len(x_train.columns)}')



features: Index(['avg_diff_5_fouls', 'avg_diff_5_offsides',
       'avg_diff_5_ball_possession_%', 'avg_diff_5_yellow_cards',
       'avg_diff_5_red_cards', 'avg_diff_5_goalkeeper_saves', 'avg_diff_fouls',
       'avg_diff_offsides', 'avg_diff_yellow_cards', 'avg_diff_red_cards',
       'avg_diff_goalkeeper_saves'],
      dtype='object')
number of features: 11


<h1>Feature Selection</h1>
<ol>
<li>Sequential forward selection (SFS)</li>
<li>Logistic Regression as the learning algorithm</li>
<li>Objective function is the classwise ECE for the calibration branch, while accuracy is the objective function for the accuracy branch</li>
<li>The input to the process is feature subset A</li>
<li>Must define training set and evaluation set. Cross validation, with 25% of the dataset used at any one time</li>

<li>For the calibration
branch, the feature subset under which the LR model achieves
the lowest classwise-ECE (fitted to an initial training set and
evaluated on a validation set) is considered the optimal feature
subset for calibration and is denoted as subset B. </li>
<li>For the accuracy branch, the feature subset under which
the highest accuracy is achieved is considered optimal and is
denoted as subset C.</li>
<li>For calibration, there must be at least 16 out of 20 bins filled</li>
</ol>

In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

def expected_calibration_error(samples=None, true_labels=None, confidences=None, accuracies=None, M=10):
    if samples is not None and true_labels is not None:
        confidences = np.max(samples, axis=1)
        predicted_label = np.argmax(samples, axis=1)
        accuracies = predicted_label == true_labels
    elif confidences is not None and accuracies is not None:
        if len(confidences) != len(accuracies):
            raise ValueError("confidences and accuracies must have the same length.")
    else:
        raise ValueError("Either (samples and true_labels) or (confidences and accuracies) must be provided.")
    
    bin_boundaries = np.linspace(0, 1, M + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

    ece = 0.0
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        in_bin = np.logical_and(confidences > bin_lower, confidences <= bin_upper)
        prob_in_bin = np.mean(in_bin)
        if prob_in_bin > 0:
            accuracy_in_bin = np.mean(accuracies[in_bin])
            avg_confidence_in_bin = np.mean(confidences[in_bin])
            ece += np.abs(avg_confidence_in_bin - accuracy_in_bin) * prob_in_bin
    return ece

def select_calibration_features(X, y):
    current_features = []
    remaining_features = X.columns.tolist()
    best_ece = float('inf')
    M_bins = 10
    min_filled_bins = 5
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    while remaining_features:
        candidates = []
        for feature in remaining_features:
            candidate_features = current_features + [feature]
            X_subset = X[candidate_features]
            
            all_confidences = []
            all_accuracies = []
            
            for train_idx, val_idx in kf.split(X_subset):
                X_train_fold = X_subset.iloc[train_idx]
                X_val_fold = X_subset.iloc[val_idx]
                y_train_fold = y.iloc[train_idx]
                y_val_fold = y.iloc[val_idx]
                
                lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
                lr.fit(X_train_fold, y_train_fold)
                y_probs = lr.predict_proba(X_val_fold)
                
                fold_confidences = np.max(y_probs, axis=1)
                fold_accuracies = (np.argmax(y_probs, axis=1) == y_val_fold.to_numpy())
                
                all_confidences.extend(fold_confidences)
                all_accuracies.extend(fold_accuracies)
            
            all_confidences = np.array(all_confidences)
            all_accuracies = np.array(all_accuracies)
            
            filled_bins = sum(
                np.logical_and(all_confidences > (i/M_bins), all_confidences <= ((i+1)/M_bins)).any()
                for i in range(M_bins)
            )
            
            if filled_bins >= min_filled_bins:
                ece = expected_calibration_error(confidences=all_confidences, accuracies=all_accuracies, M=M_bins)
                candidates.append((ece, feature))
        
        if not candidates:
            break
            
        candidates.sort(key=lambda x: x[0])
        best_ece_candidate, best_feature = candidates[0]
        
        if best_ece_candidate < best_ece:
            best_ece = best_ece_candidate
            current_features.append(best_feature)
            remaining_features.remove(best_feature)
        else:
            break
    
    return current_features

def select_accuracy_features(X, y):
    features = X.columns.tolist()
    remaining_features = features.copy()
    selected_features = []
    best_accuracy = 0.0
    best_feature_set = []
    TOLERANCE = 0.001
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    while remaining_features:
        current_best_accuracy = best_accuracy - TOLERANCE
        current_best_feature = None
        
        for feature in remaining_features:
            candidate_features = selected_features + [feature]
            X_subset = X[candidate_features]
            
            fold_accuracies = []
            for train_idx, val_idx in kf.split(X_subset):
                X_train_fold = X_subset.iloc[train_idx]
                X_val_fold = X_subset.iloc[val_idx]
                y_train_fold = y.iloc[train_idx]
                y_val_fold = y.iloc[val_idx]
                
                lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
                lr.fit(X_train_fold, y_train_fold)
                y_pred = lr.predict(X_val_fold)
                acc = accuracy_score(y_val_fold, y_pred)
                fold_accuracies.append(acc)
            
            avg_acc = np.mean(fold_accuracies)
            
            if avg_acc > current_best_accuracy:
                current_best_accuracy = avg_acc
                current_best_feature = feature
        
        if current_best_feature:
            selected_features.append(current_best_feature)
            remaining_features.remove(current_best_feature)
            best_accuracy = current_best_accuracy
            best_feature_set = selected_features.copy()
        else:
            break
    
    print(f'Best cross-validated accuracy: {best_accuracy}')
    return best_feature_set

def feature_selection(input_df, input_labels):
    le = LabelEncoder()
    y_encoded = le.fit_transform(input_labels)
    y = pd.Series(y_encoded)
    
    calibration_features = select_calibration_features(input_df, y)
    accuracy_features = select_accuracy_features(input_df, y)
    
    return calibration_features, accuracy_features

calib_features, acc_features = feature_selection(x_feature_selection_tuning, y_feature_selection_tuning)


print(f'Calibration features: {calib_features}')
print(f'Number of calibration features: {len(calib_features)}')
print(f'Accuracy features: {acc_features}')
print(f'Number of accuracy features: {len(acc_features)}')

from sklearn.feature_selection import mutual_info_classif
mi = mutual_info_classif(x_feature_selection_tuning[acc_features], y_feature_selection_tuning)
print(pd.Series(mi, index=x_feature_selection_tuning[acc_features].columns).sort_values())



c:\Users\yongh\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\yongh\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\yongh\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\yongh\AppData\Local\Programs\Python\P

Best cross-validated accuracy: 0.5303433575296693
Calibration features: ['avg_diff_goalkeeper_saves', 'avg_diff_yellow_cards', 'avg_diff_red_cards', 'avg_diff_fouls']
Number of calibration features: 4
Accuracy features: ['avg_diff_5_ball_possession_%', 'avg_diff_offsides', 'avg_diff_5_red_cards', 'avg_diff_5_fouls', 'avg_diff_goalkeeper_saves', 'avg_diff_5_yellow_cards', 'avg_diff_red_cards', 'avg_diff_5_offsides']
Number of accuracy features: 8
avg_diff_offsides               0.000000
avg_diff_5_fouls                0.000000
avg_diff_red_cards              0.007890
avg_diff_5_offsides             0.008523
avg_diff_5_red_cards            0.009626
avg_diff_5_yellow_cards         0.042559
avg_diff_goalkeeper_saves       0.052222
avg_diff_5_ball_possession_%    0.067120
dtype: float64


<h1>Hyperparameter Tuning of Models using Grid Search</h1>
<ol>
<li>Logistic regression</li>
<li>Random Forest</li>
<li>SVC (Support Vector Classification)</li>
<li>XGBoost</li>
</ol>

For Accuracy

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, log_loss, accuracy_score  # Added: imported accuracy_score
import pickle


# Define search spaces
logistic_regression_space = [
    Categorical(["none", "l1", "l2", "elasticnet"], name="penalty"),
    Real(0.01, 10, name="C"),
    Categorical(["liblinear", "saga"], name="solver"),
    Real(0.0, 1.0, name="l1_ratio"),
]

svc_space = [
    Categorical(["linear", "rbf"], name="kernel"), 
    Real(0.1, 10, name="C", prior='log-uniform'),   
    Categorical(["scale", "auto"], name="gamma"),   
]
random_forest_space = [
    Integer(10, 200, name="n_estimators"),
    Integer(2, 20, name="max_depth"),
    Categorical(["gini", "entropy"], name="criterion"),
]

xgboost_space = [
    Integer(50, 100, name='n_estimators'),
    Integer(3, 5, name='max_depth'),
    Real(0.01, 0.1, name='learning_rate'),
    Real(0.7, 0.9, name='subsample'),
    Real(0.05, 0.1, name='gamma'),
    Real(0, 1, name='reg_lambda'),
    Real(0, 1, name='reg_alpha'),
]

# Define the objective function
def objective_function(params, model_type, X_train, y_train, X_test, y_test):
    param_names = [param.name for param in search_space]
    params_dict = dict(zip(param_names, params))

    if model_type == "logistic_regression":
        penalty = params_dict["penalty"]
        solver = params_dict["solver"]

        if penalty == "l1" and solver not in ["liblinear", "saga"]:
            return 1.0
        if penalty == "elasticnet" and solver != "saga":
            return 1.0
        if penalty == "none" and solver not in ["newton-cg", "lbfgs", "sag", "saga"]:
            return 1.0

        if penalty != "elasticnet":
            params_dict.pop("l1_ratio", None)

        model = LogisticRegression(**params_dict, random_state=42)

    elif model_type == "svc":
        model = SVC(**params_dict, probability=True, random_state=42)

    elif model_type == "random_forest":
        model = RandomForestClassifier(**params_dict, random_state=42)

    elif model_type == "xgboost":
        model = XGBClassifier(**params_dict, eval_metric='error', random_state=42)

    else:
        raise ValueError("Invalid model type.")

    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)  # Fixed: added y_proba calculation
        f1 = f1_score(y_test, y_pred, average='weighted')
        loss = log_loss(y_test, y_proba)
        print(f"Model: {model_type}, F1 Score: {f1:.4f}, Log Loss: {loss:.4f}")
        return -f1
    except:
        return 1.0  # Return poor score for failed fits

# Run Bayesian optimization for each model
models = {
    "logistic_regression": logistic_regression_space,
    "svc": svc_space,
    "random_forest": random_forest_space,
    "xgboost": xgboost_space,
}

results = {}

# Perform initial training and testing
for model_type, search_space in models.items():
    print(f"\nRunning optimization for {model_type}...")
    x_tuning_train, x_tuning_test, y_tuning_train, y_tuning_test = train_test_split(
        x_feature_selection_tuning[acc_features],
        y_feature_selection_tuning,
        test_size=0.2,
        random_state=42,
        shuffle=True
    )
    result = gp_minimize(
            func=lambda params: objective_function(params, model_type, 
                                                x_tuning_train, y_tuning_train, x_tuning_test, y_tuning_test),
            dimensions=search_space,
            n_calls=50,
            random_state=42,
            verbose=False
        )
    # Save the results
    results[model_type] = result
    print(f"Best hyperparameters for {model_type}: {result.x}")
    print(f"Best F1 score for {model_type}: {-result.fun:.4f}\n")

# Evaluate initial training and testing performance
for model_type, result in results.items():
    print(f"\nEvaluating initial {model_type} model...")
    param_names = [param.name for param in models[model_type]]
    best_params = dict(zip(param_names, result.x))

    # Handle special cases
    if model_type == "logistic_regression":
        if best_params.get("penalty") != "elasticnet":
            best_params.pop("l1_ratio", None)
        model = LogisticRegression(**best_params, random_state=42)
    elif model_type == "svc":
        model = SVC(**best_params, probability=True, random_state=42)
    elif model_type == "random_forest":
        model = RandomForestClassifier(**best_params, random_state=42)
    elif model_type == "xgboost":
        model = XGBClassifier(**best_params, eval_metric='error', random_state=42)

    # Train on initial training data
    model.fit(x_train[acc_features], y_train)
    
    # Get the F1 score, accuracy, and log loss on the training data
    y_pred = model.predict(x_train[acc_features])
    y_proba = model.predict_proba(x_train[acc_features])
    f1 = f1_score(y_train, y_pred, average='weighted')
    accuracy = accuracy_score(y_train, y_pred)  # Added: accuracy calculation
    loss = log_loss(y_train, y_proba)

    print(f"Training - F1 Score: {f1:.4f}, Accuracy: {accuracy:.4f}, Log Loss: {loss:.4f}")  # Added: accuracy to print

    # Evaluate on testing data
    for dataset_name, (X_data, y_data) in [("Testing", (x_test[acc_features], y_test))]:
        y_pred = model.predict(X_data)
        y_proba = model.predict_proba(X_data)
        f1 = f1_score(y_data, y_pred, average='weighted')
        accuracy = accuracy_score(y_data, y_pred)  # Added: accuracy calculation
        loss = log_loss(y_data, y_proba)
        print(f"{dataset_name} - F1 Score: {f1:.4f}, Accuracy: {accuracy:.4f}, Log Loss: {loss:.4f}")  # Added: accuracy to print

# Final training with the same hyperparameters and features
for model_type, result in results.items():
    print(f"\nTraining final {model_type} model...")
    param_names = [param.name for param in models[model_type]]
    best_params = dict(zip(param_names, result.x))

    # Handle special cases
    if model_type == "logistic_regression":
        if best_params.get("penalty") != "elasticnet":
            best_params.pop("l1_ratio", None)
        model = LogisticRegression(**best_params, random_state=42)
    elif model_type == "svc":
        model = SVC(**best_params, probability=True, random_state=42)
    elif model_type == "random_forest":
        model = RandomForestClassifier(**best_params, random_state=42)
    elif model_type == "xgboost":
        model = XGBClassifier(**best_params, eval_metric='error', random_state=42)

    # Train on final training data
    model.fit(x_final_train[acc_features], y_final_train)

    print(f"Final {model_type} model trained.")
    # Save the final model
    with open(f'../pkl_files/{model_type}_model_f1.pkl', 'wb') as f:
        pickle.dump(model, f)



Running optimization for logistic_regression...
Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0043
Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0025
Model: logistic_regression, F1 Score: 0.4322, Log Loss: 0.9985
Model: logistic_regression, F1 Score: 0.4670, Log Loss: 1.0030
Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0026
Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0046
Model: logistic_regression, F1 Score: 0.4720, Log Loss: 1.0124
Model: logistic_regression, F1 Score: 0.4629, Log Loss: 1.0034
Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0047
Model: logistic_regression, F1 Score: 0.4629, Log Loss: 1.0033
Model: logistic_regression, F1 Score: 0.4629, Log Loss: 1.0034
Model: logistic_regression, F1 Score: 0.4670, Log Loss: 1.0030
Model: logistic_regression, F1 Score: 0.4570, Log Loss: 0.9987
Model: logistic_regression, F1 Score: 0.4629, Log Loss: 1.0034
Model: logistic_regression, F1 Score: 0.4503, Log Loss: 0.9998


c:\Users\yongh\AppData\Local\Programs\Python\Python313\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [np.str_('l2'), 0.01, np.str_('liblinear'), 1.0] before, using random point ['elasticnet', 4.184180116237682, 'saga', 0.9422630642039865]
  warnings.warn(


Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0044
Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0047


c:\Users\yongh\AppData\Local\Programs\Python\Python313\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [np.str_('l2'), 0.01, np.str_('liblinear'), 1.0] before, using random point ['l1', 7.855630004066022, 'saga', 0.4372644175952557]
  warnings.warn(


Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0045


...

Model: logistic_regression, F1 Score: 0.4629, Log Loss: 1.0034


c:\Users\yongh\AppData\Local\Programs\Python\Python313\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [np.str_('l2'), 0.01, np.str_('liblinear'), 1.0] before, using random point ['elasticnet', 9.12014649714738, 'saga', 0.8593373518094732]
  warnings.warn(


Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0046


c:\Users\yongh\AppData\Local\Programs\Python\Python313\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [np.str_('l2'), 0.01, np.str_('liblinear'), 1.0] before, using random point ['none', 7.530006843442342, 'liblinear', 0.7732312746025826]
  warnings.warn(


Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0033
Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0036
Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0047
Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0046
Model: logistic_regression, F1 Score: 0.4710, Log Loss: 1.0031
Model: logistic_regression, F1 Score: 0.4670, Log Loss: 1.0030
Model: logistic_regression, F1 Score: 0.4670, Log Loss: 1.0031
Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0047
Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0046
Model: logistic_regression, F1 Score: 0.4670, Log Loss: 1.0032
Model: logistic_regression, F1 Score: 0.4629, Log Loss: 1.0034
Model: logistic_regression, F1 Score: 0.4515, Log Loss: 0.9986
Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0041
Model: logistic_regression, F1 Score: 0.4588, Log Loss: 1.0047
Model: logistic_regression, F1 Score: 0.4629, Log Loss: 1.0033
Model: logistic_regression, F1 Score: 0.4629, Log Loss:

In [ ]:
import json
import pandas as pd

# Dictionary to store all evaluation metrics
evaluation_metrics = {
    'model': [],
    'f1_score': [],
    'accuracy': [],
    'log_loss': [],
    'best_hyperparameters': []
}

# Collect metrics during the evaluation phase
for model_type, result in results.items():
    print(f"\nEvaluating initial {model_type} model...")
    param_names = [param.name for param in models[model_type]]
    best_params = dict(zip(param_names, result.x))

    # Handle special cases for model creation
    if model_type == "logistic_regression":
        if best_params.get("penalty") != "elasticnet":
            best_params.pop("l1_ratio", None)
        model = LogisticRegression(**best_params, random_state=42)
    elif model_type == "svc":
        model = SVC(**best_params, probability=True, random_state=42)
    elif model_type == "random_forest":
        model = RandomForestClassifier(**best_params, random_state=42)
    elif model_type == "xgboost":
        model = XGBClassifier(**best_params, eval_metric='error', random_state=42)

    # Train on initial training data
    model.fit(x_train[acc_features], y_train)
    
    # Get metrics on training data
    y_pred_train = model.predict(x_train[acc_features])
    y_proba_train = model.predict_proba(x_train[acc_features])
    f1_train = f1_score(y_train, y_pred_train, average='weighted')
    accuracy_train = accuracy_score(y_train, y_pred_train)
    loss_train = log_loss(y_train, y_proba_train)

    print(f"Training - F1 Score: {f1_train:.4f}, Accuracy: {accuracy_train:.4f}, Log Loss: {loss_train:.4f}")

    # Get metrics on testing data
    y_pred_test = model.predict(x_test[acc_features])
    y_proba_test = model.predict_proba(x_test[acc_features])
    f1_test = f1_score(y_test, y_pred_test, average='weighted')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    loss_test = log_loss(y_test, y_proba_test)

    print(f"Testing - F1 Score: {f1_test:.4f}, Accuracy: {accuracy_test:.4f}, Log Loss: {loss_test:.4f}")

    # Store metrics in dictionary
    evaluation_metrics['model'].append(model_type)
    evaluation_metrics['f1_score'].append({
        'training': f1_train,
        'testing': f1_test,
        'optimization_best': -result.fun
    })
    evaluation_metrics['accuracy'].append({
        'training': accuracy_train,
        'testing': accuracy_test
    })
    evaluation_metrics['log_loss'].append({
        'training': loss_train,
        'testing': loss_test
    })
    evaluation_metrics['best_hyperparameters'].append(best_params)

# Create filename with LLM name

filename = f"f1_tune_baseline"

# Save as JSON file
project_root = Path.cwd().parent
out_dir = project_root / "model_performance_results" / filename
out_dir.mkdir(parents=True, exist_ok=True)
json_filename = out_dir/f"{filename}.json"
with open(json_filename, 'w') as f:
    json.dump(evaluation_metrics, f, indent=4, default=str)  # default=str handles non-serializable objects
print(f"Metrics saved to {json_filename}")

# Alternative: Save as CSV file for easier analysis
csv_filename = out_dir/f"{filename}.csv"
metrics_df = pd.DataFrame({
    'Model': evaluation_metrics['model'],
    'F1_Train': [m['training'] for m in evaluation_metrics['f1_score']],
    'F1_Test': [m['testing'] for m in evaluation_metrics['f1_score']],
    'F1_Best_Opt': [m['optimization_best'] for m in evaluation_metrics['f1_score']],
    'Accuracy_Train': [m['training'] for m in evaluation_metrics['accuracy']],
    'Accuracy_Test': [m['testing'] for m in evaluation_metrics['accuracy']],
    'LogLoss_Train': [m['training'] for m in evaluation_metrics['log_loss']],
    'LogLoss_Test': [m['testing'] for m in evaluation_metrics['log_loss']]
})

metrics_df.to_csv(csv_filename, index=False)
print(f"Metrics also saved to {csv_filename}")

# Alternative: Save hyperparameters separately
hyperparams_filename = out_dir/f"hyperparams_baseline.json"
hyperparams_dict = {}
for i, model_type in enumerate(evaluation_metrics['model']):
    hyperparams_dict[model_type] = evaluation_metrics['best_hyperparameters'][i]

with open(hyperparams_filename, 'w') as f:
    json.dump(hyperparams_dict, f, indent=4, default=str)
print(f"Hyperparameters saved to {hyperparams_filename}")



Evaluating initial logistic_regression model...
Training - F1 Score: 0.4596, Accuracy: 0.5338, Log Loss: 0.9990
Testing - F1 Score: 0.4630, Accuracy: 0.5333, Log Loss: 1.0074

Evaluating initial svc model...
Training - F1 Score: 0.4489, Accuracy: 0.5260, Log Loss: 1.0004
Testing - F1 Score: 0.4620, Accuracy: 0.5364, Log Loss: 1.0138

Evaluating initial random_forest model...
Training - F1 Score: 0.4745, Accuracy: 0.5555, Log Loss: 0.9359
Testing - F1 Score: 0.4229, Accuracy: 0.5061, Log Loss: 1.0072

Evaluating initial xgboost model...
Training - F1 Score: 0.4824, Accuracy: 0.5594, Log Loss: 0.9842
Testing - F1 Score: 0.4185, Accuracy: 0.4909, Log Loss: 1.0297
Metrics saved to f1_tune_baseline.json
Metrics also saved to f1_tune_baseline.csv
Hyperparameters saved to hyperparams_baseline.json


Calibration Branch

In [11]:
'''

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.calibration import calibration_curve
import pickle

# Define search spaces
logistic_regression_space = [
    Categorical(["l1", "l2"], name="penalty"),  
    Real(0.01, 10, name="C"),
    Categorical(["liblinear", "saga"], name="solver"),
    Real(0.0, 1.0, name="l1_ratio"),
]

svc_space = [
    Categorical(["linear", "rbf"], name="kernel"), 
    Real(0.1, 10, name="C", prior='log-uniform'),   
    Categorical(["scale", "auto"], name="gamma"),   
]
random_forest_space = [
    Integer(10, 200, name="n_estimators"),
    Integer(2, 20, name="max_depth"),
    Categorical(["gini", "entropy"], name="criterion"),
]

xgboost_space = [
    Integer(50, 100, name='n_estimators'),
    Integer(3, 5, name='max_depth'),
    Real(0.01, 0.1, name='learning_rate'),
    Real(0.7, 0.9, name='subsample'),
    Real(0.05, 0.1, name='gamma'),
    Real(0, 1, name='reg_lambda'),
    Real(0, 1, name='reg_alpha'),
]

# Define the objective function
def objective_function(params, model_type, X_train, y_train, X_test, y_test, search_space):
    param_names = [param.name for param in search_space]
    params_dict = dict(zip(param_names, params))


    if model_type == "logistic_regression":
        penalty = params_dict.get("penalty")
        solver = params_dict.get("solver")

        # Enhanced validation logic
        valid_combinations = {
            "liblinear": ["l1", "l2"],
            "saga": ["l1", "l2", "elasticnet", None]
        }
        if solver in valid_combinations:
            if penalty not in valid_combinations[solver]:
                return 1.0  # Penalize invalid combinations
        else:
            return 1.0  # Invalid solver

        # Handle l1_ratio parameter
        if penalty != "elasticnet":
            params_dict.pop("l1_ratio", None)

        try:
            model = LogisticRegression(**params_dict, max_iter=1000)
        except ValueError:
            return 1.0  # Catch any unexpected parameter errors

    elif model_type == "svc":
        model = SVC(**params_dict, probability=True)

    elif model_type == "random_forest":
        model = RandomForestClassifier(**params_dict)

    elif model_type == "xgboost":
        model = XGBClassifier(**params_dict, use_label_encoder=False, eval_metric='logloss')

    else:
        raise ValueError("Invalid model type.")

    model.fit(X_train, y_train)
    y_proba = model.predict_proba(X_test)
    loss = log_loss(y_test, y_proba)
    return loss  # Minimize log loss

# Function to compute Class-Wise Expected Calibration Error (ECE)
def compute_classwise_ece(y_true, y_proba, bins=10):
    """
    Compute ECE for each class separately.
    Args:
        y_true: True labels (array-like).
        y_proba: Predicted probabilities (array-like, shape [n_samples, n_classes]).
        bins: Number of bins for calibration curve.
    Returns:
        classwise_ece: Dictionary mapping class index to its ECE.
    """
    classwise_ece = {}
    n_classes = y_proba.shape[1]
    
    for class_idx in range(n_classes):
        # One-vs-rest for the current class
        y_true_binary = (y_true == class_idx).astype(int)
        y_proba_class = y_proba[:, class_idx]
        
        # Compute calibration curve
        prob_true, prob_pred = calibration_curve(y_true_binary, y_proba_class, n_bins=bins)
        
        # Compute ECE for this class
        ece = np.mean(np.abs(prob_true - prob_pred))
        classwise_ece[class_idx] = ece
    
    return classwise_ece

# Function to compute Total Class-Wise ECE (Weighted Average)
def compute_total_classwise_ece(y_true, y_proba, bins=10):
    """
    Compute the total class-wise ECE as a weighted average.
    Args:
        y_true: True labels (array-like).
        y_proba: Predicted probabilities (array-like, shape [n_samples, n_classes]).
        bins: Number of bins for calibration curve.
    Returns:
        total_ece: Weighted average of class-wise ECEs.
    """
    classwise_ece = compute_classwise_ece(y_true, y_proba, bins)
    n_classes = len(classwise_ece)
    total_weight = 0
    weighted_sum = 0
    
    for class_idx in range(n_classes):
        # Weight is the number of samples in the class
        weight = np.sum(y_true == class_idx)
        weighted_sum += weight * classwise_ece[class_idx]
        total_weight += weight
    
    total_ece = weighted_sum / total_weight if total_weight > 0 else 0
    return total_ece

# Run Bayesian optimization for each model
models = {
    "logistic_regression": logistic_regression_space,
    "svc": svc_space,
    "random_forest": random_forest_space,
    "xgboost": xgboost_space,
}

results = {}

# Perform initial training and testing
for model_type, search_space in models.items():
    print(f"\nRunning optimization for {model_type}...")
    x_tuning_train, x_tuning_test, y_tuning_train, y_tuning_test = train_test_split(
        x_feature_selection_tuning[calib_features],
        y_feature_selection_tuning,
        test_size=0.2,
        random_state=42,
        shuffle=True
    )
    result = gp_minimize(
        func=lambda params: objective_function(
            params, 
            model_type, 
            x_tuning_train, y_tuning_train, 
            x_tuning_test, y_tuning_test,
            search_space  # Pass current search space
        ),
        dimensions=search_space,
        n_calls=50,
        random_state=42,
        verbose=False
    )
    # Save the results
    results[model_type] = result
    print(f"Best hyperparameters for {model_type}: {result.x}")
    print(f"Best log loss for {model_type}: {result.fun:.4f}\n")

# Evaluate initial training and testing performance
for model_type, result in results.items():
    print(f"\nEvaluating initial {model_type} model...")
    param_names = [param.name for param in models[model_type]]
    best_params = dict(zip(param_names, result.x))

    # Handle special cases
    if model_type == "logistic_regression":
        # Post-hoc validation of best parameters
        penalty = best_params.get("penalty")
        solver = best_params.get("solver")
        
        valid_combinations = {
            "liblinear": ["l1", "l2"],
            "saga": ["l1", "l2", "elasticnet"]
        }
        
        if solver not in valid_combinations or penalty not in valid_combinations[solver]:
            raise ValueError(f"Invalid combination: penalty={penalty}, solver={solver}")
        if penalty != "elasticnet":
            best_params.pop("l1_ratio", None)
        model = LogisticRegression(**best_params)
    elif model_type == "svc":
        model = SVC(**best_params, probability=True)
    elif model_type == "random_forest":
        model = RandomForestClassifier(**best_params)
    elif model_type == "xgboost":
        model = XGBClassifier(**best_params, eval_metric='logloss')

    # Train on initial training data
    model.fit(x_train[calib_features], y_train)

    # Get metrics on the training data
    y_pred = model.predict(x_train[calib_features])
    y_proba = model.predict_proba(x_train[calib_features])
    accuracy = accuracy_score(y_train, y_pred)
    loss = log_loss(y_train, y_proba)
    total_ece = compute_total_classwise_ece(y_train, y_proba)
    print(f"Training Accuracy: {accuracy:.4f}, Training Log Loss: {loss:.4f}, Training Total ECE: {total_ece:.4f}")

    # Evaluate on testing data
    for dataset_name, (X_data, y_data) in [("Testing", (x_test[calib_features], y_test))]:
        y_pred = model.predict(X_data)
        y_proba = model.predict_proba(X_data)
        accuracy = accuracy_score(y_data, y_pred)
        loss = log_loss(y_data, y_proba)
        total_ece = compute_total_classwise_ece(y_data, y_proba)
        print(f"{dataset_name} Accuracy: {accuracy:.4f}, {dataset_name} Log Loss: {loss:.4f}, {dataset_name} Total ECE: {total_ece:.4f}")

# Final training with the same hyperparameters and features
for model_type, result in results.items():
    print(f"\nTraining final {model_type} model...")
    param_names = [param.name for param in models[model_type]]
    best_params = dict(zip(param_names, result.x))

    # Handle special cases
    if model_type == "logistic_regression":
        if best_params.get("penalty") != "elasticnet":
            best_params.pop("l1_ratio", None)
        model = LogisticRegression(**best_params)
    elif model_type == "svc":
        model = SVC(**best_params, probability=True)
    elif model_type == "random_forest":
        model = RandomForestClassifier(**best_params)
    elif model_type == "xgboost":
        model = XGBClassifier(**best_params, eval_metric='logloss')

    # Train on final training data
    model.fit(x_final_train[calib_features], y_final_train)

    print(f"Final {model_type} model trained.")
    # Save the final model
    with open(f'calibration_pkl_files/{model_type}_model_calib.pkl', 'wb') as f:
        pickle.dump(model, f)
        
        
'''
        

'\n\nimport pandas as pd\nimport numpy as np\nfrom sklearn.model_selection import train_test_split\nfrom skopt.space import Real, Integer, Categorical\nfrom skopt import gp_minimize\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.svm import SVC\nfrom sklearn.ensemble import RandomForestClassifier\nfrom xgboost import XGBClassifier\nfrom sklearn.metrics import accuracy_score, log_loss\nfrom sklearn.calibration import calibration_curve\nimport pickle\n\n# Define search spaces\nlogistic_regression_space = [\n    Categorical(["l1", "l2"], name="penalty"),  \n    Real(0.01, 10, name="C"),\n    Categorical(["liblinear", "saga"], name="solver"),\n    Real(0.0, 1.0, name="l1_ratio"),\n]\n\nsvc_space = [\n    Categorical(["linear", "rbf"], name="kernel"), \n    Real(0.1, 10, name="C", prior=\'log-uniform\'),   \n    Categorical(["scale", "auto"], name="gamma"),   \n]\nrandom_forest_space = [\n    Integer(10, 200, name="n_estimators"),\n    Integer(2, 20, name="max_depth"),\n 

<h1>Betting Simulation</h1>
Start with a 10000 bankroll
<ul>
<li>Fixed betting, keep placing $100 bets as long as there is a value bet identified</li>
<li>One-eighth Kelly betting, keep placing bets based on the Kelly criterion. Depends on the size of the current bankroll</li>
</ul>

This is to print the stats for each case

In [15]:
# Run each model from F1 feature selection against the betting simulation data (final test data)
# This is after the final model has been fitted

import pickle
import pandas as pd
import json

def what_to_bet(home_odds, draw_odds, away_odds, home, draw, away):
    if home_odds > home:
        return 'home'
    elif draw_odds > draw:
        return 'draw'
    elif away_odds > away:
        return 'away'
    return None

def amount_to_bet(bankroll, bookmaker_odds, model_odds):
    p = 1/model_odds
    kelly_criterion= p-((1-p)/(bookmaker_odds-1))
    
    if kelly_criterion > 0:
        return bankroll * kelly_criterion/8 #one eighth of the bankroll max bet
    else: 
        return 0

def update_bankroll(amount_to_bet, model_result, actual_result, bookmaker_odds):
    if model_result == actual_result:
        pnl = amount_to_bet*(bookmaker_odds-1)
    else:
        pnl = amount_to_bet*-1
    
    return pnl

data_to_save = {
    'kelly_f1': {  
        "SVC": [],
        "Random Forest": [],
        "Logistic Regression": [],
        "XGBoost": []
    }   
}

models_config = [
    {
        'name': 'SVC',
        'f1_file': f'../pkl_files/svc_model_f1.pkl',
        'f1_features': [acc_features, acc_features]
    },
    {
        'name': 'Random Forest',
        'f1_file': f'../pkl_files/random_forest_model_f1.pkl',
        'f1_features': [acc_features, acc_features]
    },
    {
        'name': 'Logistic Regression',
        'f1_file': f'../pkl_files/logistic_regression_model_f1.pkl',
        'f1_features': [acc_features, acc_features]
    },
    {
        'name': 'XGBoost',
        'f1_file': f'../pkl_files/xgboost_model_f1.pkl',
        'f1_features': [acc_features, acc_features]
    }
]

for model_config in models_config:
    
    print(f"Running betting simulation for {model_config['name']} model...")

    model_name = model_config['name']
    f1_file = model_config['f1_file']

    # Load F1 model from the pickle file
    with open(f1_file, 'rb') as f:
        f1_final_model = pickle.load(f)

    with open('../dataset/epl_combined/epl_combined_2023.json', 'r') as f:
        data_2023 = json.load(f)

    data_2023_odds = pd.json_normalize(data_2023)
    data_2023_odds = data_2023_odds[["home_odds", "draw_odds", "away_odds", "actual_winner"]]
    #convert home_odds, draw_odds, away_odds to float
    data_2023_odds["home_odds"] = data_2023_odds["home_odds"].astype(float)
    data_2023_odds["draw_odds"] = data_2023_odds["draw_odds"].astype(float)
    data_2023_odds["away_odds"] = data_2023_odds["away_odds"].astype(float)

    # F1 model predictions
    f1_y_probs = f1_final_model.predict_proba(x_betting_simulation[acc_features])
    f1_y_classes = f1_final_model.classes_
    f1_y_probs = 1 / f1_y_probs
    f1_y_implied_odds_df = pd.DataFrame(f1_y_probs, columns=f1_y_classes)

    #Concatenate the actual odds and the implied odds
    f1_df = pd.concat([data_2023_odds, f1_y_implied_odds_df], axis=1)

    # Rename the columns to match the betting simulation data
    f1_df.columns = ['home_odds', 'draw_odds', 'away_odds', 'actual_winner', 'home', 'draw', 'away']

    #convert to dictionary
    f1_dict = f1_df.to_dict(orient='records')

    starting_bankroll = 10000

    #Kelly betting for F1 model
    bankroll = starting_bankroll
    win_count = 0
    bet_count = 0
    betting_decisions = {'home': 0, 'draw': 0, 'away': 0, 'none': 0}

    for match in f1_dict:

        home_odds = match['home_odds']
        draw_odds = match['draw_odds']
        away_odds = match['away_odds']
        home = match['home']
        draw = match['draw']
        away = match['away']
        result = match['actual_winner']

        betting_decision = what_to_bet(home_odds, draw_odds, away_odds, home, draw, away)
        if betting_decision is None:
            betting_decisions['none'] += 1
        else:
            betting_decisions[betting_decision] += 1

        if betting_decision is not None:
            bet_count += 1
            odds_to_use = match[betting_decision+ '_odds']
            amount = amount_to_bet(bankroll, odds_to_use,match[betting_decision])
            bankroll += update_bankroll(amount, betting_decision, result, odds_to_use)
            if betting_decision == result:
                win_count += 1

        #add bankroll to the data_to_save dict
        data_to_save['kelly_f1'][model_config['name']].append(bankroll)

    print(f"Bankroll after betting on F1 model: {bankroll}")
    print(f"Win rate: {win_count / len(f1_dict)}")
    print(f"Number of bets: {bet_count}")
    print(betting_decisions)



# Save the results to a JSON file
with open(f'../betting_simulation_results/betting_simulation_results_standard.json', 'w') as f:
    json.dump(data_to_save, f, indent=4)


Running betting simulation for SVC model...
Bankroll after betting on F1 model: 15826.85545339238
Win rate: 0.2887537993920973
Number of bets: 323
{'home': 148, 'draw': 114, 'away': 61, 'none': 6}
Running betting simulation for Random Forest model...
Bankroll after betting on F1 model: 21395.57468922233
Win rate: 0.3130699088145897
Number of bets: 326
{'home': 152, 'draw': 140, 'away': 34, 'none': 3}
Running betting simulation for Logistic Regression model...
Bankroll after betting on F1 model: 20521.527392747714
Win rate: 0.2978723404255319
Number of bets: 328
{'home': 131, 'draw': 150, 'away': 47, 'none': 1}
Running betting simulation for XGBoost model...
Bankroll after betting on F1 model: 11262.891743130427
Win rate: 0.25227963525835867
Number of bets: 325
{'home': 121, 'draw': 194, 'away': 10, 'none': 4}
